# IMPORTS

In [7]:
# Imports 3third party

import time
import requests
import pandas as pd
import json
import boto3
import io
import boto3
from datetime import datetime, timedelta
import pyarrow as pa
import pyarrow.parquet as pq
from dotenv import load_dotenv



In [8]:
# IMPORTS/variables custom 
import os
import sys
sys.path.append(os.path.abspath('.'))

load_dotenv()
PATH_DIR = os.getenv("path_variable")
PROFILE_NAME = os.getenv("profile_name")
# Get the absolute path
absolute_directory = os.path.abspath(PATH_DIR)

# Add the directory to sys.path
sys.path.append(absolute_directory)


# Imports Resources
import pandas as pd
from pandas import json_normalize
from io import StringIO
from datetime import datetime, timedelta
from api_requests import Brapi_STOCKS_API, Brapi_notSTOCKS__API
#from parquet_model import write_parquet_to_s3_raw
from log_manager import S3LogManager
#from week_days import get_previous_business_days
from util import control_sucess, infer_schema, convert_to_string
from json_upload_local import write_json_to_s3
from cripto_modules import validate_coin_data
import time

# Import configs_dev in notebook_dir 
from config_dev import BASE_URL, REGION_NAME, \
    BUCKET, RAW_PREFIX, LOG_PREFIX, NAME_FUN, TOKEN_GENERATOR


In [4]:
# print("CHANGE ALL THE MOBILE VARIABLES IN config_dev.py FILE")
# print(f"{BUCKET} \n{RAW_PREFIX}\n {BASE_URL}")


# _DEV_

In [ ]:
##  Dev/_Debugging old version_

# Develop your code here



In [9]:
params = {
    'coin': 'BTC,ETC',
    'currency': 'BRL',
    'token': TOKEN_GENERATOR,
}

In [10]:
response = requests.get(BASE_URL, params=params)
 
if response.status_code == 200:
    data = response.json()
    print(data)

{'coins': [{'currency': 'BRL', 'currencyRateFromUSD': 5.1584, 'coinName': 'Bitcoin USD', 'coin': 'BTC', 'regularMarketChange': 1029.826174208, 'regularMarketPrice': 316708.95393280004, 'regularMarketChangePercent': 0.3262297, 'regularMarketDayLow': 312990.07251200004, 'regularMarketDayHigh': 318277.8555008, 'regularMarketDayRange': '312990.07251200004 - 318277.8555008', 'regularMarketVolume': 68224245025.9968, 'marketCap': 6238338574136.115, 'regularMarketTime': '2024-05-12T18:42:00.000Z', 'coinImageUrl': 'https://s2.coinmarketcap.com/static/img/coins/64x64/1.png'}, {'currency': 'BRL', 'currencyRateFromUSD': 5.1584, 'coinName': 'Ethereum Classic USD', 'coin': 'ETC', 'regularMarketChange': 0.484898730368, 'regularMarketPrice': 137.27997304320002, 'regularMarketChangePercent': 0.3544667, 'regularMarketDayLow': 136.26960755200002, 'regularMarketDayHigh': 138.71791315200002, 'regularMarketDayRange': '136.26960755200002 - 138.71791315200002', 'regularMarketVolume': 475008334.0288, 'marketCa

In [11]:
params = {
    'coin': 'BTC,ETC',
    'currency': 'BRL',
    'token': TOKEN_GENERATOR,
}


api = Brapi_notSTOCKS__API(
    url=BASE_URL,
    params=params
)

data = api.get_other_quotes()['coins']
schema = infer_schema(data)

timestamp_SP = datetime.now() - timedelta(hours=3)

In [12]:
schema

{'currency': 'string',
 'currencyRateFromUSD': 'float',
 'coinName': 'string',
 'coin': 'string',
 'regularMarketChange': 'float',
 'regularMarketPrice': 'float',
 'regularMarketChangePercent': 'float',
 'regularMarketDayLow': 'float',
 'regularMarketDayHigh': 'float',
 'regularMarketDayRange': 'string',
 'regularMarketVolume': 'float',
 'marketCap': 'float',
 'regularMarketTime': 'string',
 'coinImageUrl': 'string'}

In [15]:
from typing import Optional

from pydantic import BaseModel

class Coin(BaseModel):
    currency: str
    currencyRateFromUSD: float
    coinName: str 
    coin: str
    regularMarketChange: float
    regularMarketPrice: float
    regularMarketChangePercent: float
    regularMarketDayLow: float
    regularMarketDayHigh: float 
    regularMarketDayRange: str
    regularMarketVolume: float
    marketCap: float
    regularMarketTime: str
    coinImageUrl: Optional[str] = None

def validate_coin_data(data):

    data_dicts = []

    for result in data:
        coin = Coin(**result)
        
        data_dicts.append(coin.dict())

    return data_dicts

In [17]:
check_json = validate_coin_data(data)


### Lambda fun model below : Uncode it to dev

In [5]:

# V0 inicial structure for logging ect
def lambda_handler(event, context):
    inicio = time.perf_counter()
    control = "OK"
    timestamp_SP = datetime.now() - timedelta(hours=3)
    timestamp_SP_api_pull = timestamp_SP - timedelta(days=1)
    
    # loop df ... [!WARNING] if dont use it change in row 58
    #df_append = pd.DataFrame()

    # api class for requests
    params = {
    'coin': 'BTC,ETC',
    'currency': 'BRL',
    'token': TOKEN_GENERATOR,
    }


    api = Brapi_notSTOCKS__API(
    url=BASE_URL,
    params=params
    )

    data = api.get_other_quotes()['coins']
    schema = infer_schema(data)
    
    # --------------------Api requests wra wrangling
    # ------------------- Use Try and execept

    #for date in get_previous_business_days(5):
        #json_result = api_request_instance.get_routes(date)
    try:
        check_json = validate_coin_data(data)
        
    # ----------------- execept + control_var for logs management
    except Exception as e:
        control = f'Error msg: {e}'
        print(
            "{project_name}: The request returned 0 items. Original message: {msg}".format(
                msg=e
            )
        )

                            
        # except Exception as e:
        #     control = f'Error msg: {e}'
        #     print("Lighthouse - Chile - Beetrack - API returned 0 lines")

    # -------------------------- usual final strucuture for uploading and logging the ingestion.
    
    sucess_var = control_sucess(control)

    file_s3= 0
    if control == "OK":
        file_s3, file_size = write_json_to_s3(check_json, BUCKET, RAW_PREFIX, timestamp_SP, PROFILE_NAME)
        print(f"LIGHTHOUSE: Json file saved to S3 at: \n {file_s3}")
        print(f"LIGHTHOUSE: Json file size: {file_size} bytes")

    fim = time.perf_counter()
    log_manager = S3LogManager(BUCKET, LOG_PREFIX, control)
    time_var = round((fim - inicio),2)
    log_manager.add_log_entry(NAME_FUN, sucess_var, time_var, control, file_size)

# ! lambda_handler('','') !


### TESTING / DEBUGING IT

In [6]:
import time
import logging

start_time = time.perf_counter()

try:
    lambda_handler(None, None)
    logging.basicConfig(filename = "scrapper.log", level=logging.INFO, format='%(asctime)s - %(message)s', datefmt = '%Y-%m-%d %H:%M:%S')
    end = time.perf_counter()
    time = end - start_time
    logging.info(f"{NAME_FUN} - success - time {time:.2f}.")

except Exception as e:
    logging.error(f"Error.{NAME_FUN}:{e}")

LIGHTHOUSE: Json file saved to S3 at: 
 s3://aws-serveless-project-lambda-teste/dev/raw/brapi_api_cripto/date_ingestion=20240512/file.json
LIGHTHOUSE: Json file size: 1143 bytes
